In [1]:
import numpy as np
import math as m
from matplotlib import pyplot as plt

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display


def step_slice(lst, step):
    return lst[step]


def animate_list(lst, play=False, interval=200):
    slider = widgets.IntSlider(min=0, max=len(lst) - 1, step=1, value=0)
    if play:
        play_widjet = widgets.Play(interval=interval, max=len(lst) - 1)
        widgets.jslink((play_widjet, 'value'), (slider, 'value'))
        display(play_widjet)
    return interact(step_slice,
                    lst=fixed(lst),
                    step=slider)

In [3]:
def flux_limiter(r):
    
    # van Leer
    # return (r + np.abs(r)) / (1 + np.abs(r))
    
    # van Albada 1
    # return (r*r + r) / (r*r + 1)

    # van Albada 2
    # return 2*r / (r**2 + 1)
    
    # minmod
    return max(0,min(1,r))
    
def numerical_solution(mu, u, tau, h, step, a, scheme = 'upwind'):
    
    """
    :param mu: граничное условие
    :param u: значение в предыдущий момент времени
    :param tau: шаг временой сетки
    :param h: шаг кординатной сетки
    :param step: текущий момент времени
    :param a: скорость
    :param scheme: используемая для решения схема
    :return: значения в текущий момент времени 
    """
    
    c = (tau * a / (h))
    
    if (scheme == 'upwind'):
        u_new = [mu(step*tau)]  + [c*(u[i-1]  - u[i]) + u[i] for i in range(1, len(u))]
        
    elif(scheme == 'lax'):
        u = u + [u[-1]]
        u_new = [mu(step*tau)]  + [0.5*c*c*(u[i+1] -2*u[i] + u[i-1])+ 0.5*c*(u[i-1]  - u[i+1]) + u[i]
                                   for i in range(1, len(u)-1)]
    elif(scheme == 'laxtdv'):
        u = u + [u[-1]]
        u_new = [mu(step*tau), 
                 0.5*c*c*(u[2] - 2 * u[1] + u[0]) - 0.5*c*(u[2] - u[0])
                 + u[1]] + [ u[i] + c*(
                                         u[i-1] + flux_limiter(
                                                               (u[i-1] - u[i-2]) / (u[i] - u[i-1]) 
                                                               if u[i] - u[i-1] != 0 else 1  
                                                              ) 
                                        * (0.5 * (u[i-1] * (c + 1) + u[i] * (1 - c)) - u[i - 1]) 
                                        -  u[i] - flux_limiter(
                                                               (u[i] - u[i-1]) / (u[i+1] - u[i]) 
                                                               if u[i+1] - u[i] != 0 else 1
                                                              ) 
                                        * (0.5 * (u[i] * (c + 1) + u[i + 1] * (1 - c)) - u[i])
                                     ) 
                          for i in range(2, len(u)-1)]
        
    else:
        raise(Exception('Unknown scheme. Use scheme = "upwind", "lax" or "laxtdv"'))
        
    return u_new

In [4]:
#ut = (a**2) * ux, x_min < x < x_max, t_min < t
# u_x_0 = phi
# u_0_t = mu

In [5]:
speed = 1.5 #a**2
x_min = 0.0
x_max = 4.5
t_max = (x_max - 1.5) / speed 
t_min = 0

C = 0.7
h_cur_1 = 0.001
t_cur_1 = h_cur_1*C / speed

h_cur_2 = h_cur_1*10
t_cur_2 = t_cur_1*10

def step_func(x, x0, eps):
    xi = np.abs((x - x0) / eps)
    return np.heaviside(1.0 - xi, 0.0)


def parabola(x, x0, eps):
    xi = np.abs((x - x0) / eps)
    return (1.0 - xi ** 2) * np.heaviside(1.0 - xi, 0.0)


def exp_func(x, x0, eps):
    xi = np.abs((x - x0) / eps)
    return np.exp(-xi ** 2 / np.abs(1.0 - xi ** 2)) * np.heaviside(1.0 - xi, 0.0)


def sin_func(x, x0, eps):
    xi = np.abs((x - x0) / eps)
    return np.cos(0.5 * np.pi * xi) ** 3 * np.heaviside(1.0 - xi, 0.0)


x0 = 0.5
t0 = 0.8
eps = 0.45

case = 2 # Выбор граничных и начальных условий

if case == 1:
    phi = lambda x: step_func(x, x0=x0, eps=eps)
    mu =  lambda t: step_func(t, x0=t0, eps=eps)
elif case == 2:
    phi = lambda x: parabola(x, x0=x0, eps=eps)
    mu =  lambda t: parabola(t, x0=t0, eps=eps)
elif case == 3:
    phi = lambda x: exp_func(x, x0=x0, eps=eps)
    mu =  lambda t: exp_func(t, x0=t0, eps=eps)
elif case == 4:
    phi = lambda x: sin_func(x, x0=x0, eps=eps)
    mu =  lambda t: sin_func(t, x0=t0, eps=eps)


def exact(x, t):
    res = np.zeros_like(x)
    idx1 = np.nonzero(x < speed * t)
    res[idx1] = mu(t - x[idx1] / speed)
    idx2 = np.nonzero(x >= speed * t)
    res[idx2] = phi(x[idx2] - speed * t)
    return res



In [6]:
%%time

scheme = 'laxtdv' # Выбор схемы

xk_cur_1 = np.arange(x_min, x_max + h_cur_1, h_cur_1)
tn_cur_1 = np.arange(t_min, t_max + t_cur_1, t_cur_1)

xk_cur_2 = np.arange(x_min, x_max + h_cur_2, h_cur_2)
tn_cur_2 = np.arange(t_min, t_max + t_cur_2, t_cur_2)

y1_ = [phi(i) for i in xk_cur_1]
ypol1 = []

y2_ = [phi(i) for i in xk_cur_2]
ypol2 = []


plots= []

err1 =[m.sqrt(np.mean(np.square(exact(xk_cur_1,0) - y1_)))]
err2 = [m.sqrt(np.mean(np.square(exact(xk_cur_2,0) - y2_)))]

err1_max =[max(exact(xk_cur_1,0) - y1_)]
err2_max = [max(exact(xk_cur_2,0) - y2_)]

fig, ax = plt.subplots(2, 1, figsize=(10, 10))

ax[0].plot(xk_cur_2, y2_, 'r', label="h = 10^-2")
ax[0].plot(xk_cur_1, y1_, 'b', label="h = 10^-3")
ax[0].plot(xk_cur_1, exact(xk_cur_1,0), 'g', label="exact")
ax[0].legend()
ax[0].set_xlabel('xk')
ax[0].set_ylabel('u')
ax[0].set_title('0.000')
ax[0].set_ylim([-0.2, 1.2])
ax[0].set_xlim([0, 4.5])

ax[1].plot([0], err2, 'r', label="h = 10^-2, mse")
ax[1].plot([0], err1, 'b', label="h = 10^-3, mse")

ax[1].plot([0], err2_max, 'm', label="h = 10^-2, max")
ax[1].plot([0], err1_max, 'c', label="h = 10^-3, max")

ax[1].legend()
ax[1].set_title('Погрешности')
ax[1].set_xlabel('Время, t')
#ax[1].set_ylim([0, 1])
ax[1].set_xlim([0, 2])

plt.close(fig)
plots.append(fig)

for step in range(1, len(tn_cur_1)):
    

    ypol1 = numerical_solution(mu, y1_, t_cur_1, h_cur_1, step, speed, scheme) #10^-3
    y1_ = ypol1.copy()

    
    if (step %10 == 0):

        ypol2 = numerical_solution(mu, y2_, t_cur_2, h_cur_2, step/10, speed, scheme) #10^-2
        y2_ = ypol2.copy()
        
        err2_max.append(np.max(np.abs(exact(xk_cur_2,t_cur_2*step/10) - y2_)))
        err1_max.append(np.max(np.abs(exact(xk_cur_1,t_cur_1*step) - y1_)))
        err2.append(m.sqrt(np.mean(np.square(exact(xk_cur_2,t_cur_2*step/10) - y2_))))
        err1.append(m.sqrt(np.mean(np.square(exact(xk_cur_1,t_cur_1*step) - y1_))))
        
        fig, ax = plt.subplots(2, 1, figsize=(10, 10))

        ax[0].plot(xk_cur_2, ypol2, 'r', label="h = 10^-2")
        ax[0].plot(xk_cur_1, ypol1, 'b', label="h = 10^-3")
        ax[0].plot(xk_cur_1, exact(xk_cur_1,t_cur_1*step), 'g', label="exact")
        ax[0].legend()
        ax[0].set_xlabel('xk')
        ax[0].set_ylabel('u')
        ax[0].set_title('{:.2f}'.format(step*t_cur_1))
        ax[0].set_ylim([-0.2, 1.2])
        ax[0].set_xlim([0, 4.5])
        

        ax[1].plot([i*t_cur_2 for i in range(len(err2))], err2, 'r', label="h = 10^-2, mse")
        ax[1].plot([i*t_cur_2 for i in range(len(err1))], err1, 'b', label="h = 10^-3, mse")
        
        ax[1].plot([i*t_cur_2 for i in range(len(err2_max))], err2_max, 'm', label="h = 10^-2, max")
        ax[1].plot([i*t_cur_2 for i in range(len(err1_max))], err1_max, 'c', label="h = 10^-3, max")

        
        ax[1].legend()
        ax[1].set_title('Погрешности')
        ax[1].set_xlabel('Время, t')
        #ax[1].set_ylim([0, 1])
        ax[1].set_xlim([0, 2])
        
        plt.close(fig)
        plots.append(fig)
print('отношение max погрешностей mse:', max(err2)/max(err1))
print('отношение max погрешностей max:', max(err2_max)/max(err1_max))

отношение max погрешностей mse: 9.222863015144524
отношение max погрешностей max: 4.404525743414116
Wall time: 2min 2s


In [7]:
animate_list(plots, play=True, interval=100)

Play(value=0, max=428)

interactive(children=(IntSlider(value=0, description='step', max=428), Output()), _dom_classes=('widget-intera…

<function __main__.step_slice(lst, step)>